# Section: BLAS/LAPACK - Linear Equations


Adapted from: [https://github.com/gjbex/Fortran-MOOC/tree/master/source_code/blas_lapack/linear_equations](https://github.com/gjbex/Fortran-MOOC/tree/master/source_code/blas_lapack/linear_equations)

## This program demonstrates solving linear algebra equations in Fortran.

### Linear Equation Problem

In this notebook we will use Fortran to solve a system of linear equations of the form:

$$
\Large A \mathbf{x} = \mathbf{b}
$$
where: <br>


$$
\Large A =
\left[
\begin{array}{cccc}
a_{11} & a_{12} & \cdots & a_{1n} \\
a_{21} & a_{22} & \cdots & a_{2n} \\
\vdots & \vdots & \vdots & \vdots \\
a_{m1} & a_{m2} & \cdots & a_{mn}
\end{array}
\right]
$$

$$
\Large \mathbf{x} =
\left[
\begin{array}{c}
x_{1}  \\
x_{2}  \\
\vdots \\
x_{n}
\end{array}
\right]
$$

$$
\Large \mathbf{b} = 
\left[
\begin{array}{c}
b_{1}  \\
b_{2}  \\
\vdots \\
b_{n}
\end{array}
\right]
$$

The approach will be to have a main program that reads in as parameters the number of equations, the matrix $\mathbf{A}$ and the vector $\mathbf{x}$ as text files.  The main program is called *solve_equations.f90*.  It makes use of the LAPACK linear equation solver routine called **DGESV**.  Information on this routine can be found at [LAPACK - DGESV](https://netlib.org/lapack/explore-html/d7/d3b/group__double_g_esolve_ga5ee879032a8365897c3ba91e3dc8d512.html#ga5ee879032a8365897c3ba91e3dc8d512).

The main program makes use of a module called *linalg_mod.f90*, which contains subroutines to read and write matrices and arrays to and from text files.

Finally, there is a helper program called *generate_array.f90* which has a subroutine that gets arguments from the command line and generates either of vector or matrix of specified size.  Every element in the array or matrix is a random number.

The individual program files are listed below:

### In file linalg_mod.f90

```{literalinclude} Fortran_Code/Section_BLAS_LAPACK_Linear_Equations/src/linalg_mod.f90
---
language: fortran
---
```

### In file generate_array.f90

```{literalinclude} Fortran_Code/Section_BLAS_LAPACK_Linear_Equations/app/generate_array.f90
---
language: fortran
---
```

### In solve_equations.f90

```{literalinclude} Fortran_Code/Section_BLAS_LAPACK_Linear_Equations/app/solve_equations.f90
---
language: fortran
---
```

The above programs are compiled and run using Fortran Package Manager (fpm):

## Build the Program using FPM (Fortran Package Manager)

In [1]:
import os
root_dir = ""
root_dir = os.getcwd()

Since the code makes use of the LAPACK library, the following FPM configuration file (fpm.toml) was used:

```{literalinclude} Fortran_Code/Section_BLAS_LAPACK_Linear_Equations/fpm.toml
---
language: toml
---
```

In [2]:
code_dir = root_dir + "/" + "Fortran_Code/Section_BLAS_LAPACK_Linear_Equations"

In [3]:
os.chdir(code_dir)

The files *solve_equations.f90* and *generate_array.f90* were placed into the "app" folder, while the file *linalg_mod.f90* was placed into the "src" folder.

In [4]:
build_status = os.system("fpm build 2>/dev/null")

[  0%]            solve_equations.f90
[ 50%]            solve_equations.f90  done.
[ 50%]                solve_equations
[100%]                solve_equations  done.
[100%] Project compiled successfully.


## Run the Program using FPM (Fortran Package Manager)

### Solve a Test Linear System of Two Equations

As our first run, we wish to solve the following set of linear equations:

$$
\begin{align*}
2x+8y & = 20 \\
x+2y  & = 4
\end{align*}
$$

The variables in the equations are converted into components of the $\mathbf{x}$ vector as shown below:

$$
\begin{align*}
2x_1+8x_2 & = 20 \\
x_1+2x_2  & = 4
\end{align*}
$$

These equations are converted into matrix form as shown below:

$$
\begin{equation*}
\left[
\begin{array}{cc}
2 & 8 \\
1 & 2 \\
\end{array}
\right]
\left[
\begin{array}{c}
x_1 \\
x_2 \\
\end{array}
\right]
=
\left[
\begin{array}{c}
20 \\
4 \\
\end{array}
\right]
\end{equation*}
$$

Therefore we have the following:

$$
\mathbf{A} = 
\left[
\begin{array}{cc}
2 & 8 \\
1 & 2 
\end{array}
\right]
$$

$$
\mathbf{x} = 
\left[
\begin{array}{c}
x_1 \\
x_2  
\end{array}
\right]
$$


$$
\mathbf{b} = 
\left[
\begin{array}{c}
20 \\
4  
\end{array}
\right]
$$


The matrix $\mathbf{A}$ and the vector $\mathbf{b}$ are written into text files as shown below:

In [5]:
%%writefile A_test1.txt
2 8
1 2

Overwriting A_test1.txt


In [6]:
%%writefile b_test1.txt
20
4

Overwriting b_test1.txt


The *solve_equations* program can now be run with the number of equations command line argument set to 2, and the files *A_test1.txt* and *b_test1.txt*

In [7]:
exec_status = \
    os.system("fpm run solve_equations 2>/dev/null -- 2 A_test1.txt b_test1.txt")

     -0.200000000000000E+01
      0.300000000000000E+01


The results are printed in scientfic notation and in the order of $x_1$, $x_2$.

We now wish to use Python's Numpy library to test these results:

In [8]:
import numpy as np

A = np.genfromtxt("A_test1.txt")
b = np.genfromtxt("b_test1.txt")
x = np.linalg.solve(A, b)
print("x1 = {0:2.1f}".format(x[0]))
print("x2 = {0:2.1f}".format(x[1]))

x1 = -2.0
x2 = 3.0


We can see that the Fortran code and Numpy produce the same results.

### Solve a Test Linear System of Equations of Arbitrary Size

The Fortran code can be used to solve an arbitrarily large system of equations.  To test this functionality, we make use of the *generate_array.f90* program to generate arrays or matrices of arbitrary size filled with random numbers.

As a start, we will use the *generate_array.f90* to generate a matrix file A_test2.txt that contains a 10x10 matrix.

In [9]:
exec_status = os.system("fpm run generate_array 2>/dev/null -- 10 10 > A_test2.txt") 

The $\mathbf{A}$ matrix is shown below:

In [10]:
import pandas as pd
A = pd.read_table("A_test2.txt", 
    header=None, 
    sep='\s+')
A

,0,1,2,3,4,5,6,7,8,9
0,0.562645,0.398895,0.876649,0.663628,0.399495,0.719344,0.138819,0.926478,0.200899,0.457556
1,0.836904,0.373585,0.479704,0.786975,0.190448,0.103434,0.269080,0.823230,0.449362,0.623339
2,0.887208,0.931421,0.280401,0.034176,0.099787,0.075286,0.340718,0.844026,0.920811,0.255420
3,0.546763,0.171930,0.048926,0.960883,0.807276,0.309901,0.017471,0.813582,0.505252,0.704385
4,0.907811,0.634595,0.518534,0.134447,0.699955,0.623658,0.079991,0.055857,0.075261,0.010636
5,0.665585,0.456085,0.374623,0.459388,0.680324,0.572269,0.440112,0.041323,0.106486,0.106818
6,0.385047,0.656485,0.681223,0.486273,0.920283,0.402544,0.288831,0.023419,0.902289,0.481314
7,0.499228,0.226253,0.034307,0.021683,0.102256,0.961857,0.573045,0.704338,0.074513,0.730747
8,0.782151,0.990393,0.947019,0.424337,0.980281,0.289018,0.138428,0.349381,0.526987,0.375296
9,0.088794,0.914635,0.178359,0.462044,0.264483,0.685446,0.564274,0.450637,0.711282,0.868203


And now we generate the $\mathbf{b}$ vector:

In [11]:
exec_status = os.system("fpm run generate_array 2>/dev/null -- 10 > b_test2.txt") 

The $\mathbf{b}$ vector is shown below:

In [12]:
b = pd.read_table("b_test2.txt", 
    header=None, 
    sep='\s+')
b

,0
0,0.023739
1,0.064174
2,0.489469
3,0.201429
4,0.109182
5,0.538996
6,0.948285
7,0.771191
8,0.498646
9,0.511521


We now use the *solve_equation* Fortran code to solve this linear system of equations:

In [13]:
exec_status = \
    os.system("fpm run solve_equations 2>/dev/null -- 10 A_test2.txt b_test2.txt")

     -0.180072531295291E+00
     -0.666112150838889E+00
      0.215097097890775E+00
     -0.852929874551731E+00
      0.970900684366707E+00
     -0.245021949637360E+00
      0.155136840699398E+01
      0.207720068586852E+00
      0.461372913331167E+00
      0.122644088617752E+00


And the results are compared to the output of Numpy:

In [14]:
A = np.genfromtxt("A_test2.txt")
b = np.genfromtxt("b_test2.txt")
x = np.linalg.solve(A, b)

for i in range(len(x)):
    print ("x{0:d} = {1:2.6f}".format(i+1, x[i]))

x1 = -0.180073
x2 = -0.666112
x3 = 0.215097
x4 = -0.852930
x5 = 0.970901
x6 = -0.245022
x7 = 1.551368
x8 = 0.207720
x9 = 0.461373
x10 = 0.122644


Again, we see that the results are the same.